In [1]:
import os
import sys

# List of required libraries
required_libs = ["numpy", "nltk", "spacy", "scikit-learn", "wikipedia-api", "wikipedia"]

# Function to install missing libraries
def install_packages():
    for lib in required_libs:
        try:
            __import__(lib.replace("-", "_"))  # Try importing the module
        except ImportError:
            print(f"Installing {lib}...")
            os.system(f"{sys.executable} -m pip install {lib}")  # Install missing library

# Install missing packages before running the chatbot
install_packages()

# Download spaCy model if not already installed
import spacy
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading spaCy model 'en_core_web_sm'...")
    os.system(f"{sys.executable} -m spacy download en_core_web_sm")


Installing scikit-learn...
Installing wikipedia-api...


In [ ]:
import random
import warnings
import numpy as np
import wikipedia
import nltk
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings("ignore")

# Predefined chatbot responses
responses = {
    "greeting": ["Hello!", "Hi there!", "Hey! How can I assist you today?"],
    "farewell": ["Goodbye!", "See you soon!", "Take care!"],
    "name": ["I am an AI chatbot created to assist you!", "Call me ChatBot!"],
    "default": ["I'm not sure about that. Can you rephrase?", "Interesting! Tell me more."]
}

# Sample user input and responses
corpus = [
    "hi", "hello", "hey", "good morning", "good evening",
    "bye", "goodbye", "see you later",
    "what is your name", "who are you",
    "how are you", "how is your day",
    "can you help me", "i need assistance",
    "tell me a joke", "who is albert einstein", "what is python"
]
response_list = [
    "greeting", "greeting", "greeting", "greeting", "greeting",
    "farewell", "farewell", "farewell",
    "name", "name",
    "default", "default",
    "default", "default",
    "joke", "wiki", "wiki"
]

# Additional joke responses
responses["joke"] = [
    "Why don't scientists trust atoms? Because they make up everything!",
    "Why did the chicken join a band? Because it had the drumsticks!",
    "What do you get when you cross a snowman and a vampire? Frostbite!"
]

# Convert text to vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

def get_wikipedia_answer(query):
    """Fetches a short summary from Wikipedia for factual questions."""
    try:
        return wikipedia.summary(query, sentences=2)
    except:
        return "I couldn't find anything on that. Can you ask something else?"

def get_response(user_input):
    """Finds the best response using TF-IDF and Cosine Similarity."""
    user_input = user_input.lower()
    input_vector = vectorizer.transform([user_input])
    
    # Compute similarity
    similarities = cosine_similarity(input_vector, X)
    best_match_idx = np.argmax(similarities)
    
    # If similarity is low, use default response
    if similarities[0, best_match_idx] < 0.3:
        return get_wikipedia_answer(user_input)  # Try fetching from Wikipedia

    # Return matched response
    intent = response_list[best_match_idx]
    
    if intent == "wiki":
        return get_wikipedia_answer(user_input)  # Search Wikipedia
    else:
        return random.choice(responses[intent])

def chatbot():
    print("Chatbot: Hello! Type 'bye' to exit.")
    while True:
        user_input = input("You: ").lower()
        if user_input in ["bye", "exit", "quit"]:
            print("Chatbot:", random.choice(responses["farewell"]))
            break
        print("Chatbot:", get_response(user_input))

if __name__ == "__main__":
    chatbot()


Chatbot: Hello! Type 'bye' to exit.


You:  pyhton


Chatbot: I couldn't find anything on that. Can you ask something else?


You:  python


Chatbot: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.


You:  java language


Chatbot: Java is a high-level, general-purpose, memory-safe, object-oriented programming language. It is intended to let programmers write once, run anywhere (WORA), meaning that compiled Java code can run on all platforms that support Java without the need to recompile.
